# Filings Daily Stats

We need to load in these libraries into our notebook in order to query, load, manipulate and view the data

In [ ]:
import os
import psycopg2
import pandas as pd
import csv
import matplotlib
from datetime import datetime, timedelta
from IPython.core.display import HTML

%load_ext sql
%config SqlMagic.displaylimit = 5

This will create the connection to the database and prep the jupyter magic for SQL

In [ ]:
connect_to_db = 'postgresql://' + \
                os.getenv('PG_USER', '') + ":" + os.getenv('PG_PASSWORD', '') +'@' + \
                os.getenv('PG_HOST', '') + ':' + os.getenv('PG_PORT', '5432') + '/' + os.getenv('PG_DB_NAME', '');
    
%sql $connect_to_db

Simplest query to run to ensure our libraries are loaded and our DB connection is working

In [ ]:
%%sql 
select now() AT TIME ZONE 'PST' as current_date

monthly total before running time.

In [ ]:
%%sql stat_filings_daily_completed  <<
SELECT b.identifier AS INCORPORATION_NUMBER
       , b.legal_name AS INCORPORATION_NAME
       , u.username AS FILING_USER
       , f.status 
       , f.filing_date at time zone 'utc' at time zone 'pst' AS FILING_TIMESTAMP_PST
       , f.effective_date at time zone 'utc' at time zone 'pst' AS EFFECTIVE_TIMESTAMP_PST
FROM businesses b, filings f, users u 
WHERE b.id = f.business_id 
AND b.legal_type='BC'
AND f.submitter_id=u.id 
AND date(f.filing_date at time zone 'utc' at time zone 'pst') = date(current_date-1)
ORDER BY FILING_TIMESTAMP_PST;

In [ ]:
edt = stat_filings_daily_completed.DataFrame()

Save to CSV

In [ ]:
datestr = datetime.strftime(datetime.now()-timedelta(1), '%Y-%m-%d')
filename = 'incorporation_filings_daily_stats_' + datestr +'.csv'

with open(filename, 'w') as f:
    if edt.empty:
        f.write('No Data Retrieved for ' + datestr)         
    else:         
        edt.to_csv(f, sep=',', encoding='utf-8', index=False)   

In [ ]:
%%sql stat_filings_total_completed  <<
SELECT count(*)
FROM businesses b, filings f, users u 
WHERE b.id = f.business_id 
AND b.legal_type='BC'
AND f.submitter_id=u.id 
AND date(f.filing_date at time zone 'utc' at time zone 'pst') <= date(current_date-1);

In [ ]:
with open(filename, 'a') as f:  
    f.write('\n\n\n The Total Amount of Incorps to Date:\n')
    stat_filings_total_completed.DataFrame().to_csv(f, sep=',', encoding='utf-8', index=False) 